In [1]:
import os
import json
import cv2 as cv
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
from sklearn.model_selection import train_test_split
from imageai.Detection import ObjectDetection


Using TensorFlow backend.
/Users/ivan/Desktop/ALL/Soft/python3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/ivan/Desktop/ALL/Soft/python3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/ivan/Desktop/ALL/Soft/python3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/ivan/De

In [2]:

detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath("/Users/ivan/Desktop/ALL/Model/imageAI/yolo.h5")
detector.loadModel('fastest')







Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
dt = "/Users/ivan/Desktop/ALL/Data/Amap_Traffic"

In [4]:

with open(f"{dt}/amap_traffic_annotations_train.json", "r") as f:
    trai = json.load(f)

with open(f"{dt}/amap_traffic_annotations_test.json", "r") as f:
    test = json.load(f)

trai, test


({'annotations': [{'id': '000001',
    'key_frame': '3.jpg',
    'status': 0,
    'frames': [{'frame_name': '1.jpg', 'gps_time': 1552212488},
     {'frame_name': '2.jpg', 'gps_time': 1552212493},
     {'frame_name': '3.jpg', 'gps_time': 1552212498},
     {'frame_name': '4.jpg', 'gps_time': 1552212503}]},
   {'id': '000002',
    'key_frame': '4.jpg',
    'status': 0,
    'frames': [{'frame_name': '1.jpg', 'gps_time': 1555301199},
     {'frame_name': '2.jpg', 'gps_time': 1555301204},
     {'frame_name': '3.jpg', 'gps_time': 1555301209},
     {'frame_name': '4.jpg', 'gps_time': 1555301214}]},
   {'id': '000003',
    'key_frame': '5.jpg',
    'status': 0,
    'frames': [{'frame_name': '1.jpg', 'gps_time': 1552864748},
     {'frame_name': '2.jpg', 'gps_time': 1552864753},
     {'frame_name': '3.jpg', 'gps_time': 1552864758},
     {'frame_name': '4.jpg', 'gps_time': 1552864764},
     {'frame_name': '5.jpg', 'gps_time': 1552864768}]},
   {'id': '000004',
    'key_frame': '3.jpg',
    'status'

In [5]:

sample = None

inrun = True
# inrun = False


In [6]:


def feat(n1, n2, data):
    if inrun:
        rate = 10
        ty = n1
        rl = []

        for i in tqdm(data["annotations"][:sample]):
            iy, ix, _ = cv.imread(f"{dt}/{ty}/{i['id']}/{i['key_frame']}").shape
            r = {i:[None]*44 for i in range(-4, 4+1)}

            for j in i["frames"]:
                det1 = detector.detectObjectsFromImage(
                    input_image=f"{dt}/{ty}/{i['id']}/{j['frame_name']}",
                    minimum_percentage_probability=rate
                )
                det2 = [
                    [[_["percentage_probability"]] + _["box_points"]] 
                    for _ in det1 if _["name"] == "car"
                ]
                if det2:
                    det3 = np.array(det2)

                    k = int(j["frame_name"][0]) - int(i["key_frame"][0])
                    r[k] = [
                        len(det1),
                        len(det2),
                        
                        np.max(det3[:,:,0]), 
                        np.min(det3[:,:,0]), 
                        np.max(det3[:,:,0])-np.min(det3[:,:,0]), 
                        np.max(det3[:,:,0])-np.mean(det3[:,:,0]), 
                        np.max(det3[:,:,0])-np.median(det3[:,:,0]), 
                        np.mean(det3[:,:,0])-np.median(det3[:,:,0]), 
                        np.mean(det3[:,:,0]), 
                        np.median(det3[:,:,0]),
                        np.std(det3[:,:,0]),

                        np.max(det3[:,:,1]), 
                        np.min(det3[:,:,1]), 
                        np.max(det3[:,:,1])-np.min(det3[:,:,1]),
                        np.max(det3[:,:,1])-np.mean(det3[:,:,1]), 
                        np.max(det3[:,:,1])-np.median(det3[:,:,1]), 
                        np.mean(det3[:,:,1])-np.median(det3[:,:,1]), 
                        np.mean(det3[:,:,1]),
                        np.median(det3[:,:,1]), 
                        np.std(det3[:,:,1]),
                        
                        np.max(det3[:,:,2]), 
                        np.min(det3[:,:,2]), 
                        np.max(det3[:,:,2])-np.min(det3[:,:,2]), 
                        np.max(det3[:,:,2])-np.mean(det3[:,:,2]), 
                        np.max(det3[:,:,2])-np.median(det3[:,:,2]), 
                        np.mean(det3[:,:,2])-np.median(det3[:,:,2]), 
                        np.mean(det3[:,:,2]), 
                        np.median(det3[:,:,2]), 
                        np.std(det3[:,:,2]),
                        
                        np.max(det3[:,:,3]), 
                        np.min(det3[:,:,3]), 
                        np.max(det3[:,:,3])-np.min(det3[:,:,3]), 
                        np.max(det3[:,:,3])-np.mean(det3[:,:,3]), 
                        np.max(det3[:,:,3])-np.median(det3[:,:,3]), 
                        np.mean(det3[:,:,3])-np.median(det3[:,:,3]), 
                        np.mean(det3[:,:,3]), 
                        np.median(det3[:,:,3]), 
                        np.std(det3[:,:,3]),
                        
                        np.max(det3[:,:,4]), 
                        np.min(det3[:,:,4]), 
                        np.max(det3[:,:,4])-np.min(det3[:,:,4]), 
                        np.max(det3[:,:,4])-np.mean(det3[:,:,4]), 
                        np.max(det3[:,:,4])-np.median(det3[:,:,4]), 
                        np.mean(det3[:,:,4])-np.median(det3[:,:,4]), 
                        np.mean(det3[:,:,4]), 
                        np.median(det3[:,:,4]), 
                        np.std(det3[:,:,4]),

                        abs(np.max(det3[:,:,1])/ix-0.5), 
                        abs(np.min(det3[:,:,1])/ix-0.5), 
                        abs(np.mean(det3[:,:,1])/ix-0.5),
                        abs(np.median(det3[:,:,1])/ix-0.5),
                        
                        abs(np.max(det3[:,:,2])/iy-0.5), 
                        abs(np.min(det3[:,:,2])/iy-0.5), 
                        abs(np.mean(det3[:,:,2])/iy-0.5),
                        abs(np.median(det3[:,:,2])/iy-0.5),
                        
                        abs(np.max(det3[:,:,3])/ix-0.5), 
                        abs(np.min(det3[:,:,3])/ix-0.5), 
                        abs(np.mean(det3[:,:,3])/ix-0.5),
                        abs(np.median(det3[:,:,3])/ix-0.5),
                        
                        abs(np.max(det3[:,:,4])/iy-0.5), 
                        abs(np.min(det3[:,:,4])/iy-0.5), 
                        abs(np.mean(det3[:,:,4])/iy-0.5),
                        abs(np.median(det3[:,:,4])/iy-0.5),
                        
                        j["gps_time"]
                    ]
            rl.append([i["id"], i["status"]]+r[-4]+r[-3]+r[-2]+r[-1]+r[0]+r[1]+r[2]+r[3]+r[4])
            # break

        prl = pd.DataFrame(rl)
        prl.to_pickle(f"{dt}/data/{n2}.pkl")


In [7]:

%time feat(n1="amap_traffic_train_0712", n2="trai", data=trai)


100%|██████████| 1500/1500 [42:18<00:00,  1.96s/it]


CPU times: user 1h 11min 35s, sys: 7min 17s, total: 1h 18min 53s
Wall time: 42min 19s


In [8]:

prl = pd.read_pickle(f"{dt}/data/trai.pkl")
print(pd.value_counts(prl[1]))
prl


0    1094
2     291
1     115
Name: 1, dtype: int64


0    1     2     3          4          5          6          7    \
0     000001    0   NaN   NaN        NaN        NaN        NaN        NaN   
1     000002    0   NaN   NaN        NaN        NaN        NaN        NaN   
2     000003    0   NaN   NaN        NaN        NaN        NaN        NaN   
3     000004    0   NaN   NaN        NaN        NaN        NaN        NaN   
4     000005    0   NaN   NaN        NaN        NaN        NaN        NaN   
5     000006    2   NaN   NaN        NaN        NaN        NaN        NaN   
6     000007    2   NaN   NaN        NaN        NaN        NaN        NaN   
7     000008    2   NaN   NaN        NaN        NaN        NaN        NaN   
8     000009    0   NaN   NaN        NaN        NaN        NaN        NaN   
9     000010    0   NaN   NaN        NaN        NaN        NaN        NaN   
10    000011    0   NaN   NaN        NaN        NaN        NaN        NaN   
11    000012    0   NaN   NaN        NaN        NaN        NaN        NaN   
12    000013    0   NaN   NaN        NaN        NaN        NaN        NaN   
13    000014    0   NaN   NaN        NaN        NaN        NaN        NaN   
14    000015    1  13.0   4.0  87.492549  13.706224  73.786326  34.946999   
15    000016    0   NaN   NaN        NaN        NaN        NaN        NaN   
16    000017    0   3.0   2.0  23.999260  11.738220  12.261041   6.130520   
17    000018    0   NaN   NaN        NaN        NaN        NaN        NaN   
18    000019    0   NaN   NaN        NaN        NaN        NaN        NaN   
19    000020    0   NaN   NaN        NaN        NaN        NaN        NaN   
20    000021    2   NaN   NaN        NaN        NaN        NaN        NaN   
21    000022    0   NaN   NaN        NaN        NaN        NaN        NaN   
22    000023    0   NaN   NaN        NaN        NaN        NaN        NaN   
23    000024    1   NaN   NaN        NaN        NaN        NaN        NaN   
24    000025    0   NaN   NaN        NaN        NaN        NaN        NaN   
25    000026    0   NaN   NaN        NaN        NaN        NaN        NaN   
26    000027    0   NaN   NaN        NaN        NaN        NaN        NaN   
27    000028    0   8.0   5.0  22.743522  10.094903  12.648618   7.634014   
28    000029    0  10.0   7.0  64.192075  11.999605  52.192470  24.318477   
29    000030    2   NaN   NaN        NaN        NaN        NaN        NaN   
...      ...  ...   ...   ...        ...        ...        ...        ...   
1470  001471    0   NaN   NaN        NaN        NaN        NaN        NaN   
1471  001472    0  10.0   8.0  93.762827  14.075071  79.687756  59.968522   
1472  001473    2   NaN   NaN        NaN        NaN        NaN        NaN   
1473  001474    0   NaN   NaN        NaN        NaN        NaN        NaN   
1474  001475    0   NaN   NaN        NaN        NaN        NaN        NaN   
1475  001476    0   NaN   NaN        NaN        NaN        NaN        NaN   
1476  001477    0   NaN   NaN        NaN        NaN        NaN        NaN   
1477  001478    0   NaN   NaN        NaN        NaN        NaN        NaN   
1478  001479    0   NaN   NaN        NaN        NaN        NaN        NaN   
1479  001480    2   NaN   NaN        NaN        NaN        NaN        NaN   
1480  001481    1  19.0  13.0  89.013213  11.783532  77.229681  48.160426   
1481  001482    0   NaN   NaN        NaN        NaN        NaN        NaN   
1482  001483    2   NaN   NaN        NaN        NaN        NaN        NaN   
1483  001484    1   NaN   NaN        NaN        NaN        NaN        NaN   
1484  001485    0   NaN   NaN        NaN        NaN        NaN        NaN   
1485  001486    0   NaN   NaN        NaN        NaN        NaN        NaN   
1486  001487    1   NaN   NaN        NaN        NaN        NaN        NaN   
1487  001488    0   NaN   NaN        NaN        NaN        NaN        NaN   
1488  001489    2   NaN   NaN        NaN        NaN        NaN        NaN   
1489  001490    0  12.0  11.0  34.264508  10.751083  23.513424  14.343073   
1490 

In [9]:

prl.fillna(-999999, inplace=True)
m_x = prl[prl.columns.drop([0, 1])]
m_y = prl[1]

X_trai, X_test, y_trai, y_test = train_test_split(m_x, m_y, test_size=0.30)
print(X_trai.shape, X_test.shape, y_trai.shape, y_test.shape)

import xgboost as xgb
d_train = xgb.DMatrix(X_trai, label=y_trai)
d_test = xgb.DMatrix(X_test, label=y_test)

params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',
    'num_class': 3,
    'eval_metric': 'mlogloss',
#     'max_depth': 6,
#     'subsample': 0.75,
#     'colsample_bytree': 0.75,
#     'min_child_weight': 2,
#     'eta': 0.1,
#     'silent': 1,
    'learning_rate': 0.001,
}

watchlist = [(d_train,'train'), (d_test, "test")]
%time bst = xgb.train(params, d_train, num_boost_round=3600, evals=watchlist)
bst.save_model("../outs/m001.model")

y_trai_p = bst.predict(xgb.DMatrix(X_trai))
y_test_p = bst.predict(xgb.DMatrix(X_test))

a_trai = metrics.accuracy_score(y_trai, y_trai_p)
f_trai = metrics.f1_score(y_trai, y_trai_p, average='macro')
a_test = metrics.accuracy_score(y_test, y_test_p)
f_test = metrics.f1_score(y_test, y_test_p, average='macro')
print(">>> AC Train:%.6f, Tests:%.6f" % (a_trai, a_test))
print(">>> F1 Train:%.6f, Tests:%.6f" % (f_trai, f_test))
# >>> F1 Train:0.963367, Tests:0.464979


(1050, 496) (450, 496) (1050,) (450,)
[0]	train-mlogloss:1.09755	test-mlogloss:1.09786
[1]	train-mlogloss:1.09649	test-mlogloss:1.09709
[2]	train-mlogloss:1.09543	test-mlogloss:1.09633
[3]	train-mlogloss:1.09438	test-mlogloss:1.09557
[4]	train-mlogloss:1.09333	test-mlogloss:1.09481
[5]	train-mlogloss:1.09227	test-mlogloss:1.09405
[6]	train-mlogloss:1.09122	test-mlogloss:1.09330
[7]	train-mlogloss:1.09018	test-mlogloss:1.09255
[8]	train-mlogloss:1.08913	test-mlogloss:1.09179
[9]	train-mlogloss:1.08808	test-mlogloss:1.09104
[10]	train-mlogloss:1.08703	test-mlogloss:1.09029
[11]	train-mlogloss:1.08600	test-mlogloss:1.08954
[12]	train-mlogloss:1.08495	test-mlogloss:1.08879
[13]	train-mlogloss:1.08391	test-mlogloss:1.08805
[14]	train-mlogloss:1.08288	test-mlogloss:1.08730
[15]	train-mlogloss:1.08184	test-mlogloss:1.08657
[16]	train-mlogloss:1.08081	test-mlogloss:1.08582
[17]	train-mlogloss:1.07978	test-mlogloss:1.08509
[18]	train-mlogloss:1.07875	test-mlogloss:1.08435
[19]	train-mlogloss:1.

[163]	train-mlogloss:0.94342	test-mlogloss:0.99065
[164]	train-mlogloss:0.94258	test-mlogloss:0.99009
[165]	train-mlogloss:0.94175	test-mlogloss:0.98953
[166]	train-mlogloss:0.94090	test-mlogloss:0.98896
[167]	train-mlogloss:0.94005	test-mlogloss:0.98840
[168]	train-mlogloss:0.93920	test-mlogloss:0.98783
[169]	train-mlogloss:0.93837	test-mlogloss:0.98729
[170]	train-mlogloss:0.93753	test-mlogloss:0.98672
[171]	train-mlogloss:0.93669	test-mlogloss:0.98617
[172]	train-mlogloss:0.93585	test-mlogloss:0.98560
[173]	train-mlogloss:0.93503	test-mlogloss:0.98505
[174]	train-mlogloss:0.93418	test-mlogloss:0.98449
[175]	train-mlogloss:0.93334	test-mlogloss:0.98393
[176]	train-mlogloss:0.93251	test-mlogloss:0.98338
[177]	train-mlogloss:0.93168	test-mlogloss:0.98282
[178]	train-mlogloss:0.93085	test-mlogloss:0.98228
[179]	train-mlogloss:0.93002	test-mlogloss:0.98172
[180]	train-mlogloss:0.92919	test-mlogloss:0.98117
[181]	train-mlogloss:0.92836	test-mlogloss:0.98062
[182]	train-mlogloss:0.92753	te

[324]	train-mlogloss:0.82329	test-mlogloss:0.91267
[325]	train-mlogloss:0.82263	test-mlogloss:0.91225
[326]	train-mlogloss:0.82197	test-mlogloss:0.91183
[327]	train-mlogloss:0.82130	test-mlogloss:0.91141
[328]	train-mlogloss:0.82063	test-mlogloss:0.91099
[329]	train-mlogloss:0.81996	test-mlogloss:0.91057
[330]	train-mlogloss:0.81930	test-mlogloss:0.91015
[331]	train-mlogloss:0.81863	test-mlogloss:0.90973
[332]	train-mlogloss:0.81797	test-mlogloss:0.90932
[333]	train-mlogloss:0.81730	test-mlogloss:0.90890
[334]	train-mlogloss:0.81665	test-mlogloss:0.90849
[335]	train-mlogloss:0.81599	test-mlogloss:0.90808
[336]	train-mlogloss:0.81533	test-mlogloss:0.90766
[337]	train-mlogloss:0.81467	test-mlogloss:0.90724
[338]	train-mlogloss:0.81401	test-mlogloss:0.90685
[339]	train-mlogloss:0.81335	test-mlogloss:0.90643
[340]	train-mlogloss:0.81270	test-mlogloss:0.90602
[341]	train-mlogloss:0.81204	test-mlogloss:0.90560
[342]	train-mlogloss:0.81140	test-mlogloss:0.90520
[343]	train-mlogloss:0.81074	te

[485]	train-mlogloss:0.72573	test-mlogloss:0.85211
[486]	train-mlogloss:0.72518	test-mlogloss:0.85176
[487]	train-mlogloss:0.72462	test-mlogloss:0.85142
[488]	train-mlogloss:0.72406	test-mlogloss:0.85107
[489]	train-mlogloss:0.72352	test-mlogloss:0.85073
[490]	train-mlogloss:0.72296	test-mlogloss:0.85039
[491]	train-mlogloss:0.72242	test-mlogloss:0.85005
[492]	train-mlogloss:0.72188	test-mlogloss:0.84972
[493]	train-mlogloss:0.72132	test-mlogloss:0.84937
[494]	train-mlogloss:0.72077	test-mlogloss:0.84903
[495]	train-mlogloss:0.72021	test-mlogloss:0.84869
[496]	train-mlogloss:0.71967	test-mlogloss:0.84836
[497]	train-mlogloss:0.71912	test-mlogloss:0.84801
[498]	train-mlogloss:0.71858	test-mlogloss:0.84768
[499]	train-mlogloss:0.71802	test-mlogloss:0.84734
[500]	train-mlogloss:0.71747	test-mlogloss:0.84700
[501]	train-mlogloss:0.71692	test-mlogloss:0.84666
[502]	train-mlogloss:0.71638	test-mlogloss:0.84633
[503]	train-mlogloss:0.71583	test-mlogloss:0.84599
[504]	train-mlogloss:0.71528	te

[646]	train-mlogloss:0.64353	test-mlogloss:0.80198
[647]	train-mlogloss:0.64308	test-mlogloss:0.80170
[648]	train-mlogloss:0.64263	test-mlogloss:0.80141
[649]	train-mlogloss:0.64215	test-mlogloss:0.80112
[650]	train-mlogloss:0.64170	test-mlogloss:0.80083
[651]	train-mlogloss:0.64123	test-mlogloss:0.80055
[652]	train-mlogloss:0.64078	test-mlogloss:0.80025
[653]	train-mlogloss:0.64031	test-mlogloss:0.79997
[654]	train-mlogloss:0.63986	test-mlogloss:0.79968
[655]	train-mlogloss:0.63940	test-mlogloss:0.79941
[656]	train-mlogloss:0.63895	test-mlogloss:0.79911
[657]	train-mlogloss:0.63848	test-mlogloss:0.79883
[658]	train-mlogloss:0.63804	test-mlogloss:0.79854
[659]	train-mlogloss:0.63757	test-mlogloss:0.79826
[660]	train-mlogloss:0.63712	test-mlogloss:0.79798
[661]	train-mlogloss:0.63668	test-mlogloss:0.79769
[662]	train-mlogloss:0.63622	test-mlogloss:0.79740
[663]	train-mlogloss:0.63577	test-mlogloss:0.79712
[664]	train-mlogloss:0.63530	test-mlogloss:0.79685
[665]	train-mlogloss:0.63486	te

...

[2865]	train-mlogloss:0.20810	test-mlogloss:0.61784
[2866]	train-mlogloss:0.20801	test-mlogloss:0.61784
[2867]	train-mlogloss:0.20793	test-mlogloss:0.61783
[2868]	train-mlogloss:0.20785	test-mlogloss:0.61782
[2869]	train-mlogloss:0.20778	test-mlogloss:0.61782
[2870]	train-mlogloss:0.20770	test-mlogloss:0.61782
[2871]	train-mlogloss:0.20762	test-mlogloss:0.61781
[2872]	train-mlogloss:0.20755	test-mlogloss:0.61781
[2873]	train-mlogloss:0.20747	test-mlogloss:0.61780
[2874]	train-mlogloss:0.20739	test-mlogloss:0.61780
[2875]	train-mlogloss:0.20731	test-mlogloss:0.61779
[2876]	train-mlogloss:0.20724	test-mlogloss:0.61779
[2877]	train-mlogloss:0.20716	test-mlogloss:0.61777
[2878]	train-mlogloss:0.20708	test-mlogloss:0.61776
[2879]	train-mlogloss:0.20700	test-mlogloss:0.61776
[2880]	train-mlogloss:0.20693	test-mlogloss:0.61776
[2881]	train-mlogloss:0.20686	test-mlogloss:0.61775
[2882]	train-mlogloss:0.20678	test-mlogloss:0.61775
[2883]	train-mlogloss:0.20669	test-mlogloss:0.61774
[2884]	train

[3023]	train-mlogloss:0.19631	test-mlogloss:0.61681
[3024]	train-mlogloss:0.19624	test-mlogloss:0.61680
[3025]	train-mlogloss:0.19618	test-mlogloss:0.61680
[3026]	train-mlogloss:0.19612	test-mlogloss:0.61679
[3027]	train-mlogloss:0.19605	test-mlogloss:0.61679
[3028]	train-mlogloss:0.19598	test-mlogloss:0.61678
[3029]	train-mlogloss:0.19591	test-mlogloss:0.61677
[3030]	train-mlogloss:0.19584	test-mlogloss:0.61677
[3031]	train-mlogloss:0.19577	test-mlogloss:0.61677
[3032]	train-mlogloss:0.19570	test-mlogloss:0.61677
[3033]	train-mlogloss:0.19564	test-mlogloss:0.61676
[3034]	train-mlogloss:0.19558	test-mlogloss:0.61676
[3035]	train-mlogloss:0.19551	test-mlogloss:0.61676
[3036]	train-mlogloss:0.19544	test-mlogloss:0.61674
[3037]	train-mlogloss:0.19538	test-mlogloss:0.61674
[3038]	train-mlogloss:0.19531	test-mlogloss:0.61674
[3039]	train-mlogloss:0.19524	test-mlogloss:0.61673
[3040]	train-mlogloss:0.19518	test-mlogloss:0.61673
[3041]	train-mlogloss:0.19510	test-mlogloss:0.61673
[3042]	train

[3181]	train-mlogloss:0.18601	test-mlogloss:0.61672
[3182]	train-mlogloss:0.18594	test-mlogloss:0.61672
[3183]	train-mlogloss:0.18587	test-mlogloss:0.61672
[3184]	train-mlogloss:0.18581	test-mlogloss:0.61673
[3185]	train-mlogloss:0.18575	test-mlogloss:0.61672
[3186]	train-mlogloss:0.18568	test-mlogloss:0.61672
[3187]	train-mlogloss:0.18562	test-mlogloss:0.61672
[3188]	train-mlogloss:0.18555	test-mlogloss:0.61673
[3189]	train-mlogloss:0.18548	test-mlogloss:0.61674
[3190]	train-mlogloss:0.18541	test-mlogloss:0.61674
[3191]	train-mlogloss:0.18533	test-mlogloss:0.61675
[3192]	train-mlogloss:0.18526	test-mlogloss:0.61675
[3193]	train-mlogloss:0.18520	test-mlogloss:0.61675
[3194]	train-mlogloss:0.18514	test-mlogloss:0.61675
[3195]	train-mlogloss:0.18508	test-mlogloss:0.61675
[3196]	train-mlogloss:0.18501	test-mlogloss:0.61676
[3197]	train-mlogloss:0.18493	test-mlogloss:0.61676
[3198]	train-mlogloss:0.18487	test-mlogloss:0.61676
[3199]	train-mlogloss:0.18479	test-mlogloss:0.61676
[3200]	train

[3339]	train-mlogloss:0.17561	test-mlogloss:0.61741
[3340]	train-mlogloss:0.17554	test-mlogloss:0.61742
[3341]	train-mlogloss:0.17548	test-mlogloss:0.61743
[3342]	train-mlogloss:0.17541	test-mlogloss:0.61744
[3343]	train-mlogloss:0.17535	test-mlogloss:0.61745
[3344]	train-mlogloss:0.17528	test-mlogloss:0.61746
[3345]	train-mlogloss:0.17521	test-mlogloss:0.61746
[3346]	train-mlogloss:0.17514	test-mlogloss:0.61747
[3347]	train-mlogloss:0.17508	test-mlogloss:0.61748
[3348]	train-mlogloss:0.17502	test-mlogloss:0.61749
[3349]	train-mlogloss:0.17495	test-mlogloss:0.61750
[3350]	train-mlogloss:0.17488	test-mlogloss:0.61751
[3351]	train-mlogloss:0.17482	test-mlogloss:0.61752
[3352]	train-mlogloss:0.17475	test-mlogloss:0.61753
[3353]	train-mlogloss:0.17468	test-mlogloss:0.61754
[3354]	train-mlogloss:0.17462	test-mlogloss:0.61755
[3355]	train-mlogloss:0.17454	test-mlogloss:0.61756
[3356]	train-mlogloss:0.17448	test-mlogloss:0.61758
[3357]	train-mlogloss:0.17441	test-mlogloss:0.61759
[3358]	train

[3497]	train-mlogloss:0.16580	test-mlogloss:0.61923
[3498]	train-mlogloss:0.16574	test-mlogloss:0.61923
[3499]	train-mlogloss:0.16568	test-mlogloss:0.61925
[3500]	train-mlogloss:0.16563	test-mlogloss:0.61926
[3501]	train-mlogloss:0.16557	test-mlogloss:0.61927
[3502]	train-mlogloss:0.16551	test-mlogloss:0.61928
[3503]	train-mlogloss:0.16546	test-mlogloss:0.61929
[3504]	train-mlogloss:0.16539	test-mlogloss:0.61930
[3505]	train-mlogloss:0.16535	test-mlogloss:0.61930
[3506]	train-mlogloss:0.16528	test-mlogloss:0.61931
[3507]	train-mlogloss:0.16523	test-mlogloss:0.61932
[3508]	train-mlogloss:0.16518	test-mlogloss:0.61933
[3509]	train-mlogloss:0.16511	test-mlogloss:0.61935
[3510]	train-mlogloss:0.16506	test-mlogloss:0.61935
[3511]	train-mlogloss:0.16500	test-mlogloss:0.61936
[3512]	train-mlogloss:0.16495	test-mlogloss:0.61937
[3513]	train-mlogloss:0.16489	test-mlogloss:0.61938
[3514]	train-mlogloss:0.16484	test-mlogloss:0.61938
[3515]	train-mlogloss:0.16478	test-mlogloss:0.61939
[3516]	train

In [10]:
# raise Exception()

In [11]:

%time feat(n1="amap_traffic_test_0712", n2="test", data=test)


100%|██████████| 600/600 [17:59<00:00,  2.18s/it]

CPU times: user 29min 25s, sys: 2min 54s, total: 32min 20s
Wall time: 17min 59s


In [12]:

prl = pd.read_pickle(f"{dt}/data/test.pkl")
print(pd.value_counts(prl[1]))
prl.fillna(-999999, inplace=True)
prl


-1    600
Name: 1, dtype: int64


0    1         2         3              4              5    \
0    000001   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
1    000002   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
2    000003   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
3    000004   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
4    000005   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
5    000006   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
6    000007   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
7    000008   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
8    000009   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
9    000010   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
10   000011   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
11   000012   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
12   000013   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
13   000014   -1      12.0       5.0      86.778909      16.416514   
14   000015   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
15   000016   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
16   000017   -1       8.0       7.0      91.038281      12.263725   
17   000018   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
18   000019   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
19   000020   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
20   000021   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
21   000022   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
22   000023   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
23   000024   -1       9.0       7.0      54.265636      12.304927   
24   000025   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
25   000026   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
26   000027   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
27   000028   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
28   000029   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
29   000030   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
..      ...  ...       ...       ...            ...            ...   
570  000571   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
571  000572   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
572  000573   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
573  000574   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
574  000575   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
575  000576   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
576  000577   -1      11.0       5.0      83.660108      18.076375   
577  000578   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
578  000579   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
579  000580   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
580  000581   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
581  000582   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
582  000583   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
583  000584   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
584  000585   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
585  000586   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
586  000587   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
587  000588   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
588  000589   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
589  000590   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
590  000591   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
591  000592   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
592  000593   -1       7.0       5.0      44.409505      11.770892   
593  000594   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
594  000595   -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
595  000596   -1 -

In [13]:

prl["score"] = bst.predict(xgb.DMatrix(prl[prl.columns.drop([0, 1])]))
print(pd.value_counts(prl["score"]))
prl


0.0    489
2.0    106
1.0      5
Name: score, dtype: int64


0  1         2         3              4              5  \
0    000001 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
1    000002 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
2    000003 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
3    000004 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
4    000005 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
5    000006 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
6    000007 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
7    000008 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
8    000009 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
9    000010 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
10   000011 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
11   000012 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
12   000013 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
13   000014 -1      12.0       5.0      86.778909      16.416514   
14   000015 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
15   000016 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
16   000017 -1       8.0       7.0      91.038281      12.263725   
17   000018 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
18   000019 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
19   000020 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
20   000021 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
21   000022 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
22   000023 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
23   000024 -1       9.0       7.0      54.265636      12.304927   
24   000025 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
25   000026 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
26   000027 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
27   000028 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
28   000029 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
29   000030 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
..      ... ..       ...       ...            ...            ...   
570  000571 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
571  000572 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
572  000573 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
573  000574 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
574  000575 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
575  000576 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
576  000577 -1      11.0       5.0      83.660108      18.076375   
577  000578 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
578  000579 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
579  000580 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
580  000581 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
581  000582 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
582  000583 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
583  000584 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
584  000585 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
585  000586 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
586  000587 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
587  000588 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
588  000589 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
589  000590 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
590  000591 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
591  000592 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
592  000593 -1       7.0       5.0      44.409505      11.770892   
593  000594 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
594  000595 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
595  000596 -1 -999999.0 -999999.0 -999999.000000 -999999.000000   
596  000597 -1 -999999.0 -999999.0 -999999.000000 -999999.000000  

In [14]:

result = {"annotations": []}
for i, j in zip(prl[0], prl["score"]):
    for _ in test["annotations"]:
        if _["id"] == i:
            _["status"] = int(j)
            result["annotations"].append(_)
result


{'annotations': [{'id': '000001',
   'key_frame': '4.jpg',
   'status': 0,
   'frames': [{'frame_name': '1.jpg', 'gps_time': 1555401017},
    {'frame_name': '2.jpg', 'gps_time': 1555401022},
    {'frame_name': '3.jpg', 'gps_time': 1555401027},
    {'frame_name': '4.jpg', 'gps_time': 1555401037},
    {'frame_name': '5.jpg', 'gps_time': 1555401122}]},
  {'id': '000002',
   'key_frame': '3.jpg',
   'status': 0,
   'frames': [{'frame_name': '1.jpg', 'gps_time': 1557307818},
    {'frame_name': '2.jpg', 'gps_time': 1557307823},
    {'frame_name': '3.jpg', 'gps_time': 1557307828},
    {'frame_name': '4.jpg', 'gps_time': 1557307833}]},
  {'id': '000003',
   'key_frame': '3.jpg',
   'status': 0,
   'frames': [{'frame_name': '1.jpg', 'gps_time': 1557475655},
    {'frame_name': '2.jpg', 'gps_time': 1557475660},
    {'frame_name': '3.jpg', 'gps_time': 1557475670},
    {'frame_name': '4.jpg', 'gps_time': 1557475675},
    {'frame_name': '5.jpg', 'gps_time': 1557475680}]},
  {'id': '000004',
   'key_

In [15]:

target = json.dumps(result, ensure_ascii=False)
with open(f"{dt}/outs/resultT001.json", 'w') as f:
    f.write(target)
